# ACSE - 4  Project 3  Morar : A Smoothed Particle Hydrodynamics Solver

## 1. Description

   ### 1.1 Algorithm

Our software uses this general class structure. The SPH implementation is in `SPH_2D.cpp`, this file uses `SPH_particle`function to generate the particle objects. The `place_points` function is the main function that runs in `SPH_2D.cpp`. It updates the particles in the `particle_list` for each time-step and writes the updated particles object in a pickle file. 

The `.vtp` file can then be used as an input to the `Paraview` software and a `.avi` video showing the particles moving in time is generated. 

Our `simulate` function calls the different time-step methods (depending on which is specified). The schemes then call the smoothing function if the time is at the 10th time step. We then call the `neighbour_iterate` function and update the acceleration and the derivative of the density using the `navier_cont` function. 

### 1.2 Flowcharts

This is the flowchart of our method below:

<img src="Figures/Flowcharts1.png" style="width: 520px">
<br><br>
The solution algorithm is written as the method *simulate* in *SPH_main*, uses many functions and returns the calculated output as a datafile, according to the diagram below.

<img src="Figures/Flowcharts2.png" style="width: 400px">

### 1.3 Output of our simulation

Because the given data volume is so huge to save, in order to load it of high efficiency, we output the data in the `.vtp` files instead of traditional csv. This method has the advantage of convenience -- it can serialize arbitrary object with no extra work., and works on a pretty broad range of Python types. It is also smart in that it will only write out any single object once, making it effective to store complex structures because it will write the pickled object in a more efficient binary format, instead of the human-readable format.
The user will get the .pkl format files in the directory. The file saves the output data used for subsequent post-processing and for animation according to the user set simulation time output interval n, which is the parameter defined in the function of simulation. When the simulation function generates the new data, it will be dumped into the file immediately. So even if the function breaks down for some reasons in the midway, we can still obtain the data, which to some extent, improves the sustainability of our software.

### 1.4 Visualisation of our simulation

First, we output the `.vtp` files as the input of the Paraview. We use the Paraview to visualise our simulation and the output file is .avi and you can find these videos in the github data visualisation folder.

## 2.  Optimisation

### 2.1 Optimising the computational time: Stencil

To make the calculation time faster, we implemented a "stencil" design to halve the time required to obtain a neighbor for each particle.The

figure below shows us what the template looks like and the bucket from which we will calculate neighboring objects.

<img src="Figures/stencil.png" style="width: 300px">

The original neighborhood search algorithm traversed each particle and each particle neighborhood to find the Stokes force contribution of each particle in each particle pair. The algorithm is very inefficient and has a cost complexity of $ O (N ^ 2) $. In order to implement a more efficient algorithm, we have learned that the contribution of the acceleration of particle i from neighbor j is completely opposite in direction, but the size of neighbor j is the same. The contribution of each particle to the density term is the same for i and j.

Using this knowledge we can implement an algorithm where we only ever have to calculate the relationship between two nearby particles once. To do this we only search for neighbouring particles in a particular stencil (see image below), and for every iteration add the force contribution to both the particles and the neighbour. After the whole loop every particle will have aggregated the force contributions from every near neighbour.

One important thing to note:

The forces are only added to the neighbouring particle if the neighbouring particle does not share the same bucket. This is to prevent double duplication of forces in the same bucket. A way we could have further optimised this is to add forces to neighbouring particles in the same bucket but only if the neighbouring particle had not already been read in the for loop.

Overall this stencil searching algorithm roughly halved our computational time for each simulation.

### 2.2 Improving accuracy

AB2 scheme gave an accurate anwser and the with a relatively good performace (it is faster than Improved Euler and more accurate). AB4 could have given an even more accurate answer but at an increased computational cost.

## 3. Issues faced

For the Lennard-Jones pottential we had to try different D parameters in order to find the optimal repulsive force.
Forward euler is not optimal in its performace, but we decided to focus on more accurate timestepping schemes.

## 4. Demonstration of the software

Detailed instructions on the software installation are found on the readme.

## 5. Checking the software output using the expected shallow water wave speed

We first processed eight different dx data, and then plotted the x-position of the peaks over time, and compared these eight images. We obtain the average speed of each data through the slope of these eight images, and then we use this average speed to compare with the actual wave speed, and conclude that the difference between the average speed and the accurate speed is not very large.

# 6. Convergence analysis

<img src="Figures/convergence.png" style="width: 520px">

In [6]:
import math
h1 = 5
h2 = 2
h = (h1+h2)/2# average initial water depth 
v_expected_high = math.sqrt(9.81*h)
print(v_expected_high)

5.859607495387383


The theoretically calculated value for the velocity of a shallow wave is around 5.85 m/s. We calculated this value using the shallow wave equation : $\sqrt{g*h} $.
The values that comes closer to this for a mesh with dx = 0.01m for the first pass. With  course grid all passes result in a simmilar value of 4.5m/s.  As the resolution of the mesh increases the accuracy of the first pass calculation becomes better.